# Test running CmdStan

## Create Model and data

In [1]:
cat > ./test_cmdstan/bernoulli.stan << EOF
data { 
  int<lower=0> N; 
  int<lower=0,upper=1> y[N];
} 
parameters {
  real<lower=0,upper=1> theta;
} 
model {
  theta ~ beta(1,1);  // uniform prior on interval 0,1
  y ~ bernoulli(theta);
}
EOF

In [2]:
cat > ./test_cmdstan/data.json << EOF
{
    "N" : 10,
    "y" : [0,1,0,0,0,0,0,0,0,1]
}
EOF

## Compile model
Reference
- [Using cmdstan makefile from another directory](https://discourse.mc-stan.org/t/using-cmdstan-makefile-from-another-directory/7162/6)

In [3]:
module load gcc
module load tbb
STAN_PATH=/data/reddylab/Kuei/exe/cmdstan

**Note**

To compile a model, use absolute path instead of relative path. The relative path does not work because it is probabibly using relative path with respect to the `$STAN_PATH`

```
$ make -C $STAN_PATH ./test_cmdstan/bernoulli
make: Entering directory `/gpfs/fs1/data/reddylab/Kuei/exe/cmdstan'
make: *** No rule to make target `test_cmdstan/bernoulli'.  Stop.
make: Leaving directory `/gpfs/fs1/data/reddylab/Kuei/exe/cmdstan'
```

In [6]:
make -C $STAN_PATH /data/reddylab/Kuei/exe/test_cmdstan/bernoulli

make: Entering directory `/gpfs/fs1/data/reddylab/Kuei/exe/cmdstan'
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I stan/lib/stan_math/lib/tbb_2020.3/include   -O3 -I src -I stan/src -I lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.3.9 -I stan/lib/stan_math/lib/boost_1.75.0 -I stan/lib/stan_math/lib/sundials_5.7.0/include    -DBOOST_DISABLE_ASSERTS         -c -MT stan/src/stan/model/model_header.hpp.gch -MT stan/src/stan/model/model_header.d -MM -E -MG -MP -MF stan/src/stan/model/model_header.d stan/src/stan/model/model_header.hpp
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I stan/lib/stan_math/lib/tbb_2020.3/include   -O3 -I src -I stan/src -I lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.3.9 -I stan/lib/stan_math/lib/boost_1.75.0 -I stan/lib/stan_math/lib/sundials_5.7.0/include    -DBOOST_DISABLE_ASSERT

In [7]:
ls test_cmdstan

bernoulli  bernoulli.hpp  bernoulli.stan  data.json


## Restart the notebook and run the stan model

Restart notebook to show that to run the model, you do not need to load tbb module. You just need to load gcc

In [1]:
module load gcc

In [2]:
./test_cmdstan/bernoulli sample data file=./test_cmdstan/data.json output file=./test_cmdstan/output.csv

method = sample (Default)
  sample
    num_samples = 1000 (Default)
    num_warmup = 1000 (Default)
    save_warmup = 0 (Default)
    thin = 1 (Default)
    adapt
      engaged = 1 (Default)
      gamma = 0.050000000000000003 (Default)
      delta = 0.80000000000000004 (Default)
      kappa = 0.75 (Default)
      t0 = 10 (Default)
      init_buffer = 75 (Default)
      term_buffer = 50 (Default)
      window = 25 (Default)
    algorithm = hmc (Default)
      hmc
        engine = nuts (Default)
          nuts
            max_depth = 10 (Default)
        metric = diag_e (Default)
        metric_file =  (Default)
        stepsize = 1 (Default)
        stepsize_jitter = 0 (Default)
id = 0 (Default)
data
  file = ./test_cmdstan/data.json
init = 2 (Default)
random
  seed = 2918639267 (Default)
output
  file = ./test_cmdstan/output.csv
  diagnostic_file =  (Default)
  refresh = 100 (Default)
  sig_figs = -1 (Default)
  profile_file = profile.csv (Default)


Gradient evaluation took 3.3e-05 se

In [3]:
ls test_cmdstan

bernoulli  bernoulli.hpp  bernoulli.stan  data.json  output.csv


In [4]:
STAN_PATH=/data/reddylab/Kuei/exe/cmdstan

In [5]:
$STAN_PATH/bin/stansummary ./test_cmdstan/output.csv

Inference for Stan model: bernoulli_model
1 chains: each with iter=(1000); warmup=(0); thin=(1); 1000 iterations saved.

Warmup took 0.016 seconds
Sampling took 0.024 seconds

                Mean     MCSE   StdDev     5%   50%   95%    N_Eff  N_Eff/s    R_hat

lp__            -7.3  4.4e-02     0.86   -8.9  -7.0  -6.8      385    16034     1.00
accept_stat__   0.90  4.2e-03  1.4e-01   0.57  0.97   1.0  1.2e+03  4.9e+04  1.0e+00
stepsize__      0.93      nan  4.8e-15   0.93  0.93  0.93      nan      nan      nan
treedepth__      1.4  1.6e-02  4.8e-01    1.0   1.0   2.0  9.6e+02  4.0e+04  1.0e+00
n_leapfrog__     2.3  3.3e-02  9.8e-01    1.0   3.0   3.0  9.0e+02  3.7e+04  1.0e+00
divergent__     0.00      nan  0.0e+00   0.00  0.00  0.00      nan      nan      nan
energy__         7.9  5.6e-02  1.1e+00    6.8   7.6    10  3.9e+02  1.6e+04  1.0e+00

theta           0.25  6.6e-03     0.12  0.074  0.22  0.49      355    14794      1.0

Samples were drawn using hmc with nuts.
For each paramet